In [2]:
## 질병데이터 제거

import os
import shutil
from tqdm import tqdm

target_dir = "./data_for_yolo/data"

for tra_val in os.listdir(target_dir):
    for img_lab in tqdm(os.listdir(os.path.join(target_dir, tra_val))):
        folder_path = os.path.join(target_dir, tra_val, img_lab)
        for file_name in os.listdir(folder_path):
            if file_name.startswith("V006"):
                file_path = os.path.join(folder_path, file_name)
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.10s/it]


In [4]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

results = model.train(data='D:\\Python\\work\\project\\data_for_yolo\\data.yaml', 
                      epochs=1,
                     imgsz=640,
                     batch=-1,
                     device=0,
                     patience=15,
                     )

New https://pypi.org/project/ultralytics/8.0.124 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.121  Python-3.9.13 torch-2.0.1 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\Python\work\project\data_for_yolo\data.yaml, epochs=1, patience=15, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnosti

In [6]:
import os
import zipfile
from tqdm import tqdm

def zipdir(path, ziph):
    # ziph is zipfile handle
    total_files = sum([len(files) for r, d, files in os.walk(path)])
    
    # Creating a progress bar
    progress_bar = tqdm(total=total_files, unit="file")

    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))
            progress_bar.update(1) # Updating the progress bar
    
    progress_bar.close()

dir_path = 'D:\\Python\\work\\project\\data_for_yolo\\target' # specify directory
zip_path = 'D:\\Python\\work\\project\\data_for_yolo\\target.zip' # specify the output zip file name

zipf = zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED)
zipdir(dir_path, zipf)
zipf.close()

print("Zip file created successfully!")

100%|██████████████████████████████████████████████████████████████████████████| 98476/98476 [35:10<00:00, 46.65file/s]


Zip file created successfully!


In [1]:
## 특정 폴더와 파일을 압축
import os
import zipfile
from tqdm import tqdm

def zipdir(path, ziph):
    # ziph is zipfile handle
    total_files = sum([len(files) for r, d, files in os.walk(path)])
    
    # Creating a progress bar
    progress_bar = tqdm(total=total_files, unit="file")

    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))
            progress_bar.update(1) # Updating the progress bar
    
    progress_bar.close()

def add_file_to_zip(path, ziph):
    ziph.write(path, os.path.basename(path))

dir_path = 'D:\\Python\\work\\project\\data_for_yolo\\data' # specify directory
file_path = 'D:\\Python\\work\\project\\data_for_yolo\\data.yaml' # specify single file
zip_path = 'D:\\Python\\work\\project\\data_for_yolo\\data.zip' # specify the output zip file name

zipf = zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED)

# Zip the directory
zipdir(dir_path, zipf)

# Add the single file
add_file_to_zip(file_path, zipf)

zipf.close()

print("Zip file created successfully!")

100%|██████████████████████████████████████████████████████████████████████████| 98475/98475 [40:32<00:00, 40.48file/s]


Zip file created successfully!


In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

folder_path = ".\\data_for_yolo\\data\\train\\images"
labels_path = ".\\data_for_yolo\\data\\train\\labels"

def parse_yolo_coordinates(yolo_str):
    parts = yolo_str.split()
    return int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])

def change_class_id(yolo_str, new_class_id):
    parts = yolo_str.split()
    parts[0] = str(new_class_id)
    return ' '.join(parts)

image_files = [f for f in os.listdir(folder_path) if f.endswith('.png') and ('d1' in f or 'd2' in f)]

resume_file = 'V001_tom1_43_085_d2_11_20211031_18_06072546_38102607.png'
if resume_file in image_files:
    idx = image_files.index(resume_file)
    image_files = image_files[idx:]

total_files = len(image_files)
farm_counts = {f"tom{i+1}": len([f for f in image_files if f"tom{i+1}" in f]) for i in range(6)}

for i, img_name in enumerate(image_files, start=1):
    farm_name = img_name.split('_')[1]  # assuming 'tom1' always at this position

    img_path = os.path.join(folder_path, img_name)
    txt_path = os.path.join(labels_path, img_name.replace('.png', '.txt'))

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    img_width, img_height = 1080, 1920

    with open(txt_path, 'r') as file:
        yolo_strs = file.readlines()

    for idx, yolo_str in enumerate(yolo_strs):
        fig, ax = plt.subplots(1, figsize=(12, 12))
        ax.imshow(image)

        class_id, x, y, w, h = parse_yolo_coordinates(yolo_str)
        x, y, w, h = x * img_width, y * img_height, w * img_width, h * img_height
        bbox = patches.Rectangle((x - w / 2, y - h / 2), w, h, linewidth=1, edgecolor='r', facecolor='none')

        ax.add_patch(bbox)

        plt.pause(0.01)
        plt.show()

        print(f"Processing image {i} of {total_files}, farm {farm_name} ({image_files[:i-1].count(farm_name)} of {farm_counts[farm_name]})")
        
        answer = input(f"Do you want to change class id of object {idx} in image {img_name}? (y/n): ")
        if answer.lower() == 'y':
            yolo_strs[idx] = change_class_id(yolo_strs[idx], '1') + "\n"

        plt.close()

    with open(txt_path, 'w') as file:
        file.writelines(yolo_strs)

print("Finished")